In [1]:
# Takes 3 minutes to run
import warnings
warnings.filterwarnings('ignore')
from support._model_build_func import *

Getting data from CWP database & excel files::  33%|███▎      | 1/3 [00:04<00:08,  4.32s/it]

Time Taken for Contract:1 00:04


Getting data from CWP database & excel files::  67%|██████▋   | 2/3 [00:07<00:03,  3.58s/it]

Time Taken for Contract:2 00:07


Getting data from CWP database & excel files:: 100%|██████████| 3/3 [00:10<00:00,  3.43s/it]

Time Taken for Contract:3 00:10


In [3]:
Variables =  Variables_.copy(deep=True)
# Response = Response_.copy(deep=True)
Variables_midx = Variables_midx_.copy(deep=True)
last_valid_loc = last_valid_loc_.copy(deep=True)
Variables_with_nans = Variables_with_nans_

len(Variables), Variables.shape
# len(Response),  Response.shape

(2660, 2660, (2660, 763), (2660, 7))

In [4]:
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm

yahoo_list = ['Yahoo GSPC','Yahoo XHB','Yahoo XTN','Yahoo USDCADX','Yahoo GCF','Yahoo HGF','Yahoo CLF','Yahoo GCCIX','Yahoo LBSF','Yahoo RFP','Yahoo WFG','Yahoo IFPTO','Yahoo WY','Yahoo CFPTO']
var2 = "LumberOHLCV LBAdjClose"

def get_rolling_corr(df1,var1,window):
    df = df1.copy(deep=True)
    df.sort_index(ascending=True,inplace=True)
    df =  df[var1].rolling(window=window).corr(Variables['LumberOHLCV LBAdjClose'])
    return df.sort_index(ascending=False)

def add_yahoo_index(yahoo_list,var2):
    Variables.sort_index(ascending=False, inplace=True)
    # Response.sort_index(ascending=False, inplace=True)
    final_df = Variables[yahoo_list].copy(deep=True)
    price_corr = {}
    for var1 in yahoo_list:
        price_corr[var1] = Variables[var1].corr(Variables[var2])
        # print(var1,":",Variables[var1].corr(Response[var2]))
    CorrwithLumber_df = pd.DataFrame.from_dict(price_corr, orient='index', columns=['CorrwithLumberPrice']).sort_values(by='CorrwithLumberPrice',ascending=False)

    final_variable_list = CorrwithLumber_df[CorrwithLumber_df.CorrwithLumberPrice > 0.70].CorrwithLumberPrice.index.tolist()
    final_df = Variables[final_variable_list].copy(deep=True)

    rolling_df = pd.DataFrame(index = Variables.index)
    for var in final_variable_list:
        feat_name = var+"_ret"
        rolling_df = pd.concat([rolling_df,pd.DataFrame(get_rolling_corr(final_df,var,5), columns=[var])],axis=1)
        
    # Need to add both identical lines - else doesn't work?
    rolling_df = rolling_df.replace([np.inf, -np.inf], np.nan).fillna(rolling_df.median())
    rolling_df = rolling_df.replace([np.inf, -np.inf], np.nan).fillna(rolling_df.median())


    pacf_dict = {}
    for var in final_variable_list:
        data = rolling_df[var].dropna().values
        acf, ci = sm.tsa.acf(data, alpha=0.05)
        pacf, ci = sm.tsa.pacf(data, alpha=0.05)
        pacf_dict[var] = pacf[1]   
        # pacf_dict[var] = acf[1]

    pacf_df = pd.DataFrame.from_dict(pacf_dict, orient='index', columns=['PACF'])

    rolling_df_auto = rolling_df.copy(deep=True)
    for var in final_variable_list:
        rolling_df[var] = rolling_df[var] * pacf_df.loc[var].values[0]

    # Dropping Yahoo LBSF from both
    rolling_df_auto.drop(rolling_df_auto.columns[0],axis=1, inplace=True)
    returns_df.drop(returns_df.columns[0],axis=1, inplace=True)

    YahooIndex = pd.DataFrame(data= np.diag(returns_df.dot(rolling_df_auto.T)), index = returns_df.index,columns = ['YahooIndex'])
    
    return YahooIndex
    

In [ ]:
Variables.sort_index(ascending=False, inplace=True)
# Response.sort_index(ascending=False, inplace=True)

In [ ]:
# Response["LB_Adj_Close"].head(5)
# Variables["LumberOHLCV LBAdjClose"].sort_index(ascending=False).head(5)
yahoo_list = ['Yahoo GSPC','Yahoo XHB','Yahoo XTN','Yahoo USDCADX','Yahoo GCF','Yahoo HGF','Yahoo CLF','Yahoo GCCIX','Yahoo LBSF','Yahoo RFP','Yahoo WFG','Yahoo IFPTO','Yahoo WY','Yahoo CFPTO']

In [ ]:
final_df = Variables[yahoo_list]
final_df.head(2)

Price Correlations (Lagged series with Lumber Adj close price)

In [ ]:
var2 = "LumberOHLCV LBAdjClose"
price_corr = {}
for var1 in yahoo_list:
    price_corr[var1] = Variables[var1].corr(Variables[var2])
    # print(var1,":",Variables[var1].corr(Response[var2]))
CorrwithLumber_df = pd.DataFrame.from_dict(price_corr, orient='index', columns=['CorrwithLumberPrice']).sort_values(by='CorrwithLumberPrice',ascending=False)
CorrwithLumber_df

### Only selection Variables with correlatio above 70%

In [ ]:

final_variable_list = CorrwithLumber_df[CorrwithLumber_df.CorrwithLumberPrice > 0.70].CorrwithLumberPrice.index.tolist()
final_df = Variables[final_variable_list]
final_df.head(2)

What percentage of times both the lagged variable (eg. S&P) and Lumber were up or down together?

In [ ]:
var2 ="Adj_Close_ret"
returns_df = pd.DataFrame(index = Variables.index)
for var in final_variable_list:
    
    feat_name = var+"_ret"
    returns_df = pd.concat([returns_df,pd.DataFrame((np.log(Variables) - np.log(Variables.shift(-1)))[var])],axis=1)
    returns_df = Variables[var].copy(deep=True)
    # returns_df = pd.concat([returns_df,pd.DataFrame(Variables[var].pct_change())],axis=1)
    print(var,"",round((np.where([returns_df[var]>0],1,0) == np.where(Response['Adj_Close_ret']>0,1,0)).astype(int).sum()/len(Variables),4)*100)

In [ ]:
returns_df.shape

In [ ]:
# pd.concat([returns_df['Yahoo XTN'],Response.Adj_Close_ret],axis=1).iloc[:10]
# l = (np.where([returns_df[var]>0],1,0) == np.where(Response['Adj_Close_ret']>0,1,0)).tolist()

In [ ]:
corr =  returns_df.corr()
corr.style.background_gradient(cmap='coolwarm')


In [ ]:
def get_rolling_corr(df1,var1,window):
    df = df1.copy(deep=True)
    df.sort_index(ascending=True,inplace=True)
    df =  df[var1].rolling(window=window).corr(Variables['LumberOHLCV LBAdjClose'])
    return df.sort_index(ascending=False)

In [ ]:
final_df.shape

In [ ]:
rolling_df = pd.DataFrame(index = Variables.index)
for var in final_variable_list:
    feat_name = var+"_ret"
    rolling_df = pd.concat([rolling_df,pd.DataFrame(get_rolling_corr(final_df,var,5), columns=[var])],axis=1)
    
# Need to add both identical lines - else doesn't work?
rolling_df = rolling_df.replace([np.inf, -np.inf], np.nan).fillna(rolling_df.median())
rolling_df = rolling_df.replace([np.inf, -np.inf], np.nan).fillna(rolling_df.median())
print(rolling_df.shape)
rolling_df

### Calculating Partial Autocorrelation of Rolling df

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
pacf_dict = {}
for var in final_variable_list:
    data = rolling_df[var].dropna().values
    acf, ci = sm.tsa.acf(data, alpha=0.05)
    pacf, ci = sm.tsa.pacf(data, alpha=0.05)
    pacf_dict[var] = pacf[1]   
    # pacf_dict[var] = acf[1]

pacf_df = pd.DataFrame.from_dict(pacf_dict, orient='index', columns=['PACF'])
pacf_df.T

. Yahoo XHB - SPDR S&P Homebuilders ETF   
. Yahoo GSPC - S&P 500          
. Yahoo WFG - West Fraser Timber Co. Ltd.           
. Yahoo XTN - SPDR S&P Transportation ETF           
. Yahoo IFPTO - Interfor Corporation


### Multiplying all rolling correlations with respective partial autocorrelations

In [ ]:
rolling_df_auto = rolling_df.copy(deep=True)
for var in final_variable_list:
    rolling_df[var] = rolling_df[var] * pacf_df.loc[var].values[0]
rolling_df_auto.head(2)

In [ ]:
# Dropping Yahoo LBSF from both
rolling_df_auto.drop(rolling_df_auto.columns[0],axis=1, inplace=True)
returns_df.drop(returns_df.columns[0],axis=1, inplace=True)

YahooIndex = pd.DataFrame(data= np.diag(returns_df.dot(rolling_df_auto.T)), index = returns_df.index,columns = ['YahooIndex'])
YahooIndex.head(2).append(YahooIndex.tail(2))

In [ ]:
Variables['YahooIndex'] = YahooIndex['YahooIndex']

In [ ]:
Variables.head(2)

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [ ]:
Variables

In [ ]:
w = 3
s=Variables
[[*s.index[i : i + w]] for i in range(len(s) - w + 1)]

In [ ]:
returns_df.replace(np.nan, 0, inplace=True)
Yahoo_matrix = returns_df.dot(rolling_df_auto.T).replace(np.nan,0)
x = StandardScaler().fit_transform(Yahoo_matrix)
# x = rolling_df
# y = StandardScaler().fit_transform(lumber_df_ln)
Target = pd.DataFrame(['+ve' if i > 0 else '-ve' if i < 0 else '0' for i in Response.Adj_Close_ret.values])

In [ ]:
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(x)

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2','principal component 3'])

In [ ]:
finalDf = pd.concat([principalDf, Target], axis = 1)
finalDf.columns = ['principal component 1', 'principal component 2','principal component 3', 'Target']
finalDf.head(5)

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 Component PCA', fontsize = 20)

targets = ['+ve', '-ve', '0']
colors = ['r', 'g', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['Target'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1'], finalDf.loc[indicesToKeep, 'principal component 2'], c = color, s = 50)
ax.legend(targets)
ax.grid()

In [ ]:
from mpl_toolkits import mplot3d
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
fig = plt.figure(figsize = (8,8))
ax = plt.axes(projection='3d')

ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_zlabel('Principal Component 3', fontsize = 15)
ax.set_title('3 Component PCA', fontsize = 20)

targets = ['+ve', '-ve', '0']
colors = ['r', 'g', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['Target'] == target
    ax.scatter3D(finalDf.loc[indicesToKeep, 'principal component 1'], finalDf.loc[indicesToKeep, 'principal component 2'], finalDf.loc[indicesToKeep, 'principal component 3'], c = color, s = 50)
ax.legend(targets)
ax.grid()


In [ ]:
np.corrcoef(finalDf['principal component 1'].values, Response.Adj_Close_ret.values)

In [ ]:
final_df

In [ ]:
finalDf['principal component 1'].dropna()

In [ ]:
lumber_df.pct_change(-1).reset_index().drop('dt',axis=1)

In [ ]:
pd.concat([lumber_df.pct_change(-1).reset_index().drop('dt',axis=1),finalDf['principal component 1'].dropna()],axis=1).dropna().corr()

In [ ]:
pd.concat([lumber_df.pct_change(-1).reset_index().drop('dt',axis=1),finalDf['principal component 2'].dropna()],axis=1).dropna().corr()

In [ ]:
pd.concat([lumber_df.pct_change(-1).reset_index().drop('dt',axis=1),finalDf['principal component 3'].dropna()],axis=1).dropna().corr()

In [ ]:
0.044997*100